# 🏦 Loan Calculator Agent Tutorial 💰

Welcome to the **Loan Calculator Agent** tutorial, where we'll showcase how to:
1. **Initialize** a project and use the Microsoft Foundry ecosystem
2. **Create an Agent** with **Code Interpreter** capabilities
3. **Perform loan calculations** such as monthly payments, amortization schedules, and interest analysis
4. **Analyze financial data** with sample loan portfolio data
5. **Generate** financial insights with appropriate disclaimers

## Let's Dive In
We'll walk step-by-step, similar to our **Financial Services Advisor** sample, but with a focus on using **Code Interpreter** for loan calculations and financial data analysis. Let's begin!

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll start by importing libraries, loading environment variables, and initializing an **AIProjectClient**. We'll also create sample loan portfolio data for demonstration.

In [ ]:
# Import required libraries
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    CodeInterpreterTool,
    CodeInterpreterToolAuto
)

# Load environment variables
load_dotenv(Path().absolute().parent.parent / '.env')

tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize clients
credential = InteractiveBrowserCredential(tenant_id=tenant_id)
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)
openai_client = project_client.get_openai_client()
print("✅ Clients initialized")

# Create sample loan portfolio data as JSON
def create_sample_data():
    data = {
        "loan_portfolio": [
            {"loan_id": "L001", "customer": "John Smith", "loan_type": "Mortgage", "principal": 350000, "interest_rate": 6.5, "term_years": 30, "monthly_payment": 2212.24, "status": "Current"},
            {"loan_id": "L002", "customer": "Sarah Johnson", "loan_type": "Auto Loan", "principal": 45000, "interest_rate": 7.2, "term_years": 5, "monthly_payment": 894.52, "status": "Current"},
            {"loan_id": "L003", "customer": "Mike Chen", "loan_type": "Personal Loan", "principal": 25000, "interest_rate": 9.5, "term_years": 3, "monthly_payment": 801.23, "status": "Current"},
            {"loan_id": "L004", "customer": "Emily Davis", "loan_type": "Mortgage", "principal": 500000, "interest_rate": 6.25, "term_years": 15, "monthly_payment": 4293.77, "status": "Current"},
            {"loan_id": "L005", "customer": "Robert Wilson", "loan_type": "Business Loan", "principal": 150000, "interest_rate": 8.0, "term_years": 7, "monthly_payment": 2336.47, "status": "30 Days Late"},
            {"loan_id": "L006", "customer": "Lisa Anderson", "loan_type": "Home Equity", "principal": 75000, "interest_rate": 7.5, "term_years": 10, "monthly_payment": 890.64, "status": "Current"},
            {"loan_id": "L007", "customer": "David Brown", "loan_type": "Auto Loan", "principal": 32000, "interest_rate": 6.9, "term_years": 4, "monthly_payment": 764.89, "status": "60 Days Late"}
        ]
    }
    filename = "loan_portfolio.json"
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"📄 Created sample loan portfolio file: {filename}")
    return filename

sample_file = create_sample_data()

## 2. Create Loan Calculator Agent 👩‍💻
We'll create an agent with **Code Interpreter** enabled. This agent can run Python code to calculate loan payments, generate amortization schedules, and analyze financial data.

In [ ]:
# Create agent with code interpreter for loan calculations

code_interpreter_tool = CodeInterpreterTool(container=CodeInterpreterToolAuto())

loan_agent = project_client.agents.create_version(
    agent_name="loan-calculator-agent",
    definition=PromptAgentDefinition(
        model=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        instructions="""You are a loan calculator agent for a retail bank that can:
1. Calculate monthly loan payments using: M = P * [r(1+r)^n] / [(1+r)^n - 1]
   where M = monthly payment, P = principal, r = monthly interest rate, n = number of payments
2. Generate amortization schedules showing principal and interest breakdown
3. Analyze loan portfolios and calculate key metrics (total interest, NPV, risk assessment)
4. Compare different loan options and terms
5. Generate charts and visualizations using Python code
6. Always include financial disclaimers

When providing financial calculations, remind users:
- These calculations are for educational purposes only
- Actual loan terms may vary based on credit score and other factors
- Always consult with a licensed financial advisor for personal advice""",
        tools=[code_interpreter_tool]
    ),
    description="Loan calculator agent with code interpreter"
)
print(f"🎉 Created agent: {loan_agent.name} (version: {loan_agent.version})")

## 3. Loan Payment Calculation with Code Interpreter
We'll create a conversation for loan calculations. We'll input loan parameters (principal, interest rate, term), and ask the agent to calculate monthly payments, total interest, and show the amortization schedule.

In [ ]:
def calculate_loan_with_agent(agent, principal, annual_rate, term_years, loan_type="Mortgage"):
    """Calculate loan payments using the code interpreter agent with new Foundry API."""
    try:
        # Get OpenAI client for the new API
        openai_client = project_client.get_openai_client()
        
        # Create a conversation for this loan calculation
        conversation = openai_client.conversations.create()
        print(f"📝 Created conversation for loan calculation, ID: {conversation.id}")

        # Construct user message requesting loan calculation
        user_text = (
            f"Calculate the loan details for a {loan_type}:\n"
            f"Principal Amount: ${principal:,.2f}\n"
            f"Annual Interest Rate: {annual_rate}%\n"
            f"Loan Term: {term_years} years\n\n"
            "Please:\n"
            "1. Calculate the monthly payment using the standard amortization formula\n"
            "2. Calculate the total amount paid over the life of the loan\n"
            "3. Calculate the total interest paid\n"
            "4. Show the first 12 months of the amortization schedule\n"
            "5. Include appropriate financial disclaimers\n"
            "6. Use Python code to perform all calculations"
        )

        # Use the NEW responses API with agent reference
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": agent.name,
                    "version": agent.version
                }
            },
            input=user_text
        )
        
        print(f"🤖 Loan calculation completed!")
        if response.output_text:
            print(f"\n{response.output_text}")
        return conversation.id, response
    except Exception as e:
        print(f"❌ Error during loan calculation: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if loan_agent:
    # Example: Calculate a $350,000 mortgage at 6.5% for 30 years
    loan_conversation_id, loan_response = calculate_loan_with_agent(
        loan_agent, 
        principal=350000, 
        annual_rate=6.5, 
        term_years=30, 
        loan_type="Home Mortgage"
    )

## 4. Loan Comparison Analysis
Let's ask the agent to compare different loan options - such as a 15-year vs 30-year mortgage - to help customers make informed decisions.

In [ ]:
def compare_loan_options(agent, principal):
    """Compare different loan options using the code interpreter agent."""
    try:
        # Get OpenAI client for the new API
        openai_client = project_client.get_openai_client()
        
        # Create a conversation for loan comparison
        conversation = openai_client.conversations.create()
        print(f"📝 Created conversation for loan comparison, ID: {conversation.id}")

        # Create user message requesting loan comparison
        user_text = (
            f"Compare mortgage options for a ${principal:,.2f} home loan:\n\n"
            "Option 1: 30-year fixed at 6.5% APR\n"
            "Option 2: 15-year fixed at 5.75% APR\n"
            "Option 3: 20-year fixed at 6.25% APR\n\n"
            "Please:\n"
            "1. Calculate monthly payments for each option\n"
            "2. Calculate total interest paid for each option\n"
            "3. Show how much is saved in interest with shorter terms\n"
            "4. Create a comparison table\n"
            "5. Provide insights on which option might be better for different financial situations\n"
            "6. Include appropriate financial disclaimers\n"
            "7. Use Python code for all calculations"
        )

        # Use the NEW responses API with agent reference
        print("🔄 Processing loan comparison with agent...")
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": agent.name,
                    "version": agent.version
                }
            },
            input=user_text
        )
        
        print(f"🤖 Loan comparison completed!")
        if response.output_text:
            print(f"\n{response.output_text}")
        
        return conversation.id, response
        
    except Exception as e:
        print(f"❌ Error comparing loan options: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Test the loan comparison with a $400,000 mortgage
if loan_agent:
    print("\n🔄 Testing loan comparison with new Foundry API...")
    comparison_conversation_id, comparison_response = compare_loan_options(loan_agent, 400000)

## 5. Portfolio Risk Analysis 📊
Let's ask the agent to analyze a loan portfolio, calculate key metrics, and identify potential risks.

In [ ]:
def analyze_loan_portfolio(agent):
    """Ask the agent to analyze a loan portfolio using code interpreter."""
    try:
        # Get OpenAI client for the new API
        openai_client = project_client.get_openai_client()
        
        # Create a conversation for portfolio analysis
        conversation = openai_client.conversations.create()
        print(f"📝 Created conversation for portfolio analysis, ID: {conversation.id}")

        # Sample loan portfolio data
        portfolio_data = """
        Loan Portfolio Data:
        
        | Loan ID | Customer       | Type          | Principal  | Rate | Term | Status       |
        |---------|----------------|---------------|------------|------|------|--------------|
        | L001    | John Smith     | Mortgage      | $350,000   | 6.5% | 30yr | Current      |
        | L002    | Sarah Johnson  | Auto Loan     | $45,000    | 7.2% | 5yr  | Current      |
        | L003    | Mike Chen      | Personal Loan | $25,000    | 9.5% | 3yr  | Current      |
        | L004    | Emily Davis    | Mortgage      | $500,000   | 6.25%| 15yr | Current      |
        | L005    | Robert Wilson  | Business Loan | $150,000   | 8.0% | 7yr  | 30 Days Late |
        | L006    | Lisa Anderson  | Home Equity   | $75,000    | 7.5% | 10yr | Current      |
        | L007    | David Brown    | Auto Loan     | $32,000    | 6.9% | 4yr  | 60 Days Late |
        """

        # Create user message requesting portfolio analysis
        user_text = (
            f"Analyze this loan portfolio for a regional bank:\n\n"
            f"{portfolio_data}\n\n"
            "Please perform the following analysis using Python code:\n"
            "1. Calculate total portfolio value (sum of all principals)\n"
            "2. Calculate weighted average interest rate\n"
            "3. Calculate expected monthly revenue from all loans\n"
            "4. Identify loans at risk (late status) and calculate the value at risk\n"
            "5. Create a breakdown by loan type showing count and total value\n"
            "6. Calculate the delinquency rate (% of loans that are late)\n"
            "7. Provide risk assessment recommendations\n"
            "8. Include appropriate financial disclaimers"
        )

        # Use the NEW responses API with agent reference
        print("🔄 Processing portfolio analysis with agent...")
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": agent.name,
                    "version": agent.version
                }
            },
            input=user_text
        )
        
        print(f"🤖 Portfolio analysis completed!")
        if response.output_text:
            print(f"\n{response.output_text}")
        
        return conversation.id, response
        
    except Exception as e:
        print(f"❌ Error analyzing portfolio: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Test the portfolio analysis
if loan_agent:
    print("\n🔄 Testing portfolio risk analysis...")
    portfolio_conversation_id, portfolio_response = analyze_loan_portfolio(loan_agent)

## 6. Cleanup & Best Practices
We can remove our agent and sample data if desired. In production, you might keep them for repeated usage.

### Best Practices for Financial AI Agents
1. **Calculations** – Always show formulas, provide step-by-step breakdowns, and verify with industry-standard formulas.
2. **Disclaimers** – Include regulatory disclaimers, remind users you're not a licensed advisor.
3. **Data Handling** – Validate inputs, handle edge cases, properly sanitize sensitive financial data.
4. **Compliance** – Ensure calculations follow regulatory standards (Truth in Lending Act, APR calculations).
5. **Security** – Monitor usage, limit access to code interpreter when dealing with real customer data.

In [ ]:
def cleanup_all():
    """Clean up all created resources using new Foundry API."""
    try:
        # Delete the agent using new API (delete_version with agent_version parameter)
        if 'loan_agent' in globals() and loan_agent:
            try:
                project_client.agents.delete_version(agent_name=loan_agent.name, agent_version=loan_agent.version)
                print(f"🗑️ Deleted loan calculator agent: {loan_agent.name} (Version: {loan_agent.version})")
            except Exception as agent_error:
                print(f"⚠️ Could not delete agent: {agent_error}")

        # Delete local sample file
        if 'sample_file' in globals() and sample_file and os.path.exists(sample_file):
            try:
                os.remove(sample_file)
                print("🗑️ Deleted local loan portfolio file.")
            except Exception as local_error:
                print(f"⚠️ Could not delete local file: {local_error}")

        print("\n✅ Cleanup completed!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        import traceback
        traceback.print_exc()

# Option to run cleanup - you can comment this out if you want to keep the resources
print("🧹 Running cleanup to remove created resources...")
print("💡 Comment out the cleanup_all() call if you want to keep the agent for further testing.")
cleanup_all()

# Congratulations! 🎉

You've successfully completed the **Loan Calculator Agent** with **Code Interpreter** tutorial! Here's what you accomplished:

## ✅ **What We Built**

### **🏦 Loan Calculator Agent**
- Created an AI agent with **Code Interpreter** capabilities for financial calculations
- Enabled the agent to write and execute Python code in a secure sandbox
- Configured finance-focused instructions with appropriate regulatory disclaimers

### **📊 Key Features Demonstrated**

1. **🧮 Loan Payment Calculations**
   - Agent performed standard amortization formula calculations
   - Calculated monthly payments, total interest, and loan costs
   - Generated amortization schedules showing principal/interest breakdown
   - Provided responsible financial disclaimers

2. **📈 Loan Comparison Analysis**
   - Compared different loan terms (15-year vs 30-year mortgages)
   - Calculated interest savings for shorter terms
   - Provided insights for different financial situations
   - Created comparison tables and analysis

3. **💼 Portfolio Risk Analysis**
   - Analyzed multi-loan portfolios
   - Calculated weighted average rates and total value
   - Identified loans at risk and delinquency rates
   - Provided risk assessment recommendations

4. **🔧 Resource Management**
   - Properly cleaned up agents and local resources
   - Demonstrated best practices for resource lifecycle management

## 🎯 **Key Concepts Learned**

- **Code Interpreter Tool**: Enables agents to perform complex financial calculations
- **Amortization Formulas**: Standard loan payment calculation methods
- **Portfolio Analysis**: Risk assessment and financial metrics
- **Regulatory Compliance**: Importance of disclaimers in financial AI
- **Responsible AI**: Always include appropriate financial disclaimers

## 💡 **Best Practices Recap**

1. **Calculation Accuracy** - Always use industry-standard formulas (amortization, APR)
2. **Financial Disclaimers** - Include regulatory disclaimers in all financial advice
3. **Data Validation** - Validate inputs and handle edge cases properly
4. **Compliance** - Follow regulatory standards (Truth in Lending Act)
5. **Resource Management** - Clean up agents and files to avoid unnecessary costs
6. **Security** - Use code interpreter cautiously with real customer data

---

*Happy banking automation!* 💰🏦